### Transformacion de Movies

In [5]:
# Carga y limpieza del dataset
# Importación de librerías necesarias
import pandas as pd
df = pd.read_csv('../movies_dataset.csv', low_memory=False)

In [6]:
# Verificar la correcta combinación de datos
# Visualizo unas de las primeras columnas anidadas
print(df['genres'].head())
print(df['production_companies'].head())

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object
0       [{'name': 'Pixar Animation Studios', 'id': 3}]
1    [{'name': 'TriStar Pictures', 'id': 559}, {'na...
2    [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
3    [{'name': 'Twentieth Century Fox Film Corporat...
4    [{'name': 'Sandollar Productions', 'id': 5842}...
Name: production_companies, dtype: object


In [7]:
# Importación de librerías necesarias
import ast

# Transformo las columnas anidadas a sus estructuras correspondiente
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
df['production_companies'] = df['production_companies'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
df['production_countries'] = df['production_countries'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
df['spoken_languages'] = df['spoken_languages'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])


In [8]:

# Convierto los valores de texto a diccionarios donde sea necesario
def convert_to_dict(x):
    try:
        if pd.notnull(x) and isinstance(x, str) and len(x) > 0:
            return ast.literal_eval(x)
        else:
            return {}
    except (ValueError, SyntaxError):
        return {}

# Aplico la función de conversión a la columna
df['belongs_to_collection'] = df['belongs_to_collection'].apply(convert_to_dict)

# Luego, extraigo los valores de interés asegurándote de que x es un diccionario
df['collection_id'] = df['belongs_to_collection'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
df['collection_name'] = df['belongs_to_collection'].apply(lambda x: x.get('name') if isinstance(x, dict) else None)
df['collection_poster_path'] = df['belongs_to_collection'].apply(lambda x: x.get('poster_path') if isinstance(x, dict) else None)

In [9]:
# Desanidamos 'genres' para obetener una lista de generos como una cadena separada por comas
df['genres'] = df['genres'].apply(lambda x: ', '.join([genre['name'] for genre in x]) if x else None)

In [10]:
# Desanidamos 'production_companies' para obtener los nombres de las compañias productoras
df['production_companies'] = df['production_companies'].apply(lambda x: ', '.join([company['name'] for company in x]) if x else None)


In [11]:
# Verificar la correcta combinación de datos
# Importación de librerías necesarias
import numpy as np
# Función para convertir los valores de texto a listas de diccionarios
def convert_to_list(x):
    if isinstance(x, (np.ndarray, list)):  # Verifica si es un array o lista
        return x
    if pd.isna(x):  # Verifica si es NaN
        return []
    if isinstance(x, str) and x.strip():  # Verifica que sea una cadena no vacía
        try:
            return ast.literal_eval(x)  # Convierte el texto en una lista
        except (ValueError, SyntaxError):
            return []  # Devuelve una lista vacía si hay error en la conversión
    return []  # Devuelve una lista vacía si no cumple las condiciones anteriores

# Aplico la función para convertir los valores de la columna 'production_countries'
df['production_countries'] = df['production_countries'].apply(convert_to_list)

# Función para desanidar la lista de diccionarios en una cadena de nombres
def desanidar_lista(x):
    if isinstance(x, list):
        return ', '.join([country.get('name', '') for country in x])  # Usa .get() para evitar KeyError
    return ''  # Devuelve una cadena vacía si no es una lista

# Aplico la función para desanidar
df['production_countries'] = df['production_countries'].apply(desanidar_lista)

# Muestro las primeras filas para verificar
print(df['production_countries'].head())


0    United States of America
1    United States of America
2    United States of America
3    United States of America
4    United States of America
Name: production_countries, dtype: object


In [12]:
# Y por ultimo, desanidamos el campo 'spoken_lenguages' para obetener la lista de idiomas
df['spoken_languages'] = df['spoken_languages'].apply(lambda x: ', '.join([language['name'] for language in x]) if x else None)

In [13]:
# Convierto la columna release_date al formato de fecha
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Creo la columna release_year a partir de release_date
df['release_year'] = df['release_date'].dt.year

# Convierto release_year a int
df['release_year'] = df['release_year'].astype('Int32')  # Usa 'Int32' para manejar valores NA


In [14]:
# Verificar la correcta combinación de datos
# Convertir las columnas 'revenue' y 'budget' a numérico, forzando valores no válidos a NaN
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

# Reemplazar NaN con 0 (sin usar inplace)
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

# Crear la columna 'return' calculando el retorno
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] if row['revenue'] > 0 and row['budget'] > 0 else 0, axis=1)

# Verificar la nueva columna
print(df[['revenue', 'budget', 'return']].head())


       revenue      budget     return
0  373554033.0  30000000.0  12.451801
1  262797249.0  65000000.0   4.043035
2          0.0         0.0   0.000000
3   81452156.0  16000000.0   5.090760
4   76578911.0         0.0   0.000000


In [15]:
# Verificar la correcta combinación de datos
# Elimino las columnas no utlizadas
df = df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

# Verifico que las columnas se hayan eliminado
print(df.head())  # Muestra las primeras filas del DataFrame para confirmar


  belongs_to_collection      budget                      genres     id  \
0                    {}  30000000.0   Animation, Comedy, Family    862   
1                    {}  65000000.0  Adventure, Fantasy, Family   8844   
2                    {}         0.0             Romance, Comedy  15602   
3                    {}  16000000.0      Comedy, Drama, Romance  31357   
4                    {}         0.0                      Comedy  11862   

  original_language                                           overview  \
0                en  Led by Woody, Andy's toys live happily in his ...   
1                en  When siblings Judy and Peter discover an encha...   
2                en  A family wedding reignites the ancient feud be...   
3                en  Cheated on, mistreated and stepped on, the wom...   
4                en  Just when George Banks has recovered from his ...   

  popularity                               production_companies  \
0  21.946943                            Pix

In [16]:
# Verificar la correcta combinación de datos
#Verifico como esta quedando mi dataframe
print(df.head())
print(df.columns)

  belongs_to_collection      budget                      genres     id  \
0                    {}  30000000.0   Animation, Comedy, Family    862   
1                    {}  65000000.0  Adventure, Fantasy, Family   8844   
2                    {}         0.0             Romance, Comedy  15602   
3                    {}  16000000.0      Comedy, Drama, Romance  31357   
4                    {}         0.0                      Comedy  11862   

  original_language                                           overview  \
0                en  Led by Woody, Andy's toys live happily in his ...   
1                en  When siblings Judy and Peter discover an encha...   
2                en  A family wedding reignites the ancient feud be...   
3                en  Cheated on, mistreated and stepped on, the wom...   
4                en  Just when George Banks has recovered from his ...   

  popularity                               production_companies  \
0  21.946943                            Pix

In [17]:
# Verificar la correcta combinación de datos
# Convierto 'popularity' en un float
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')

# Convierto 'collection_id' en un int
df['collection_id'] = pd.to_numeric(df['collection_id'], errors='coerce').fillna(0).astype('int32')

# Verificamos que todos nuestros tipos de datos esten correctos
print(df.dtypes)

belongs_to_collection             object
budget                           float64
genres                            object
id                                object
original_language                 object
overview                          object
popularity                       float64
production_companies              object
production_countries              object
release_date              datetime64[ns]
revenue                          float64
runtime                          float64
spoken_languages                  object
status                            object
tagline                           object
title                             object
vote_average                     float64
vote_count                       float64
collection_id                      int32
collection_name                   object
collection_poster_path            object
release_year                       Int32
return                           float64
dtype: object


### Transformacion de credits

In [18]:
# Importación de librerías necesarias
import ast

# Cargar el archivo CSV
df2 = pd.read_csv('../credits.csv')

# Función para extraer los nombres de los actores
def extract_actor_names(cast_data):
    if isinstance(cast_data, str):
        cast_list = ast.literal_eval(cast_data)
        return [actor['name'] for actor in cast_list if 'name' in actor]
    return []

# Función para extraer el nombre del director
def extract_director_name(crew_data):
    if isinstance(crew_data, str):
        crew_list = ast.literal_eval(crew_data)
        for member in crew_list:
            if member.get('job') == 'Director':
                return member.get('name')
    return None

# Aplicar las funciones a las columnas 'cast' y 'crew'
df2['actor_names'] = df2['cast'].apply(extract_actor_names)
df2['director_name'] = df2['crew'].apply(extract_director_name)

# Seleccionar solo las columnas relevantes
df2_filtered = df2[['id', 'actor_names', 'director_name']]

# Guardar el resultado en un nuevo archivo CSV
df2_filtered.to_csv('filtered_credits.csv', index=False)

# Mostrar las primeras filas para verificar los resultados
print(df2_filtered.head())


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000002E515445A60>>
Traceback (most recent call last):
  File "C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\tomas\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1533, in enumerate
    def enumerate():
    
KeyboardInterrupt: 
